In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
import requests
import re
from datetime import datetime
import pandas as pd
import geopandas as gpd

In [3]:
pd.set_option(
    'display.max_colwidth', 100
)

In [4]:
## Get the Data

# The National Observatory of Athens has an earthquake catalog; for each year there is a txt file with all the earthquakes in Greece
noa_url = 'https://www.gein.noa.gr/HTML/Noa_cat/CAT2021.TXT'

with open('data/CAT2021.TXT', mode='w') as f:
    try:
        f.write((response := requests.get(noa_url)).text)
        response.status_code
    except Exception as e:
        print('Website currently not available')

Website currently not available


In [5]:
# The dataset contains the following columns:

with open('data/CAT2021.TXT.bak', 'r') as data:
    [print(data.readline()) for _ in range(10)]

 DATE         TIME     LAT.   LONG.  DEPTH    MAGNITUDE            

                   (GMT)    (N)    (E)    (km)       (Local)

 2021 JAN  1   00 38 24.3 38.3894 21.9832    8         1.2

 2021 JAN  1   00 57 47.9 38.3748 22.0290    8         0.8

 2021 JAN  1   01 09 25.7 38.3693 22.0290    7         1.6

 2021 JAN  1   01 28 31.0 40.4311 21.1546   17         2.1

 2021 JAN  1   01 53 23.6 34.9503 24.3283   45         1.9

 2021 JAN  1   02 05 36.1 38.3835 21.9992   10         1.2

 2021 JAN  1   02 06 34.9 38.3775 21.9868    8         1.1

 2021 JAN  1   02 07 45.8 38.3665 22.0226   14         1.1



[None, None, None, None, None, None, None, None, None, None]

In [6]:
# from text data to pandas dataframe...

with open('data/CAT2021_sub.txt', 'r', encoding='UTF-8') as f:

    # initialize 6 lists to hold the column data
    date_str = []
    time_str = []
    lat_str = []
    lon_str = []
    depth_str = []
    man_str = []

    # iterate across rows
    while (line := f.readline()):
        # regular expression to separate columns in each row
        p = re.compile(r'^(\s+\d{4}\s+[A-Z]{3}\s+\d{1,2})\s+(\d{2}\s+\d{2}\s+[\d.]+)\s+([\d.]+)\s+([\d.]+)\s+(\d+)\s+([\d.]+)$')
        # (\s+\d{4}\s+[A-Z]{3}\s+\d{1,2})  : date
        # (\d{2}\s+\d{2}\s+[\d.]+)         : time
        # ([\d.]+)                         : latitude
        # ([\d.]+)                         : longitude
        # (\d+)                            : depth
        # ([\d.]+)                         : magnitude
        
        if (vals := [i.strip() for tup in p.findall(line) for i in tup]) == []:
            # skip the first 2 lines
            continue
        else:
            date_str.append(vals[0])
            time_str.append(vals[1])
            lat_str.append(vals[2])
            lon_str.append(vals[3])
            depth_str.append(vals[4])
            man_str.append(vals[5])
    
        
    earthquakes_df = pd.DataFrame({'DATE': [datetime.strptime(d, '%Y %b %d').strftime('%d/%m/%Y') for d in date_str],
                       'TIME(GMT)': [datetime.strptime(t, '%H %M %S.%f').strftime('%H:%M:%S.%f') for t in time_str],
                       'LAT (N)': [float(lat) for lat in lat_str],
                       'LONG (E)': [float(lon) for lon in lon_str],
                       'DEPTH(km)': [int(d) for d in depth_str],
                       'MAGNITUDE(Local)': [float(m) for m in man_str]})
earthquakes_df.head()    

,DATE,TIME(GMT),LAT (N),LONG (E),DEPTH(km),MAGNITUDE(Local)
0,01/01/2021,00:38:24.300000,38.3894,21.9832,8,1.2
1,01/01/2021,00:57:47.900000,38.3748,22.0290,8,0.8
2,01/01/2021,01:09:25.700000,38.3693,22.0290,7,1.6
3,01/01/2021,01:28:31.000000,40.4311,21.1546,17,2.1
4,01/01/2021,01:53:23.600000,34.9503,24.3283,45,1.9


In [7]:
# create well known text geometry column from lat & long
earthquakes_df['wkt'] = [f"POINT ({earthquakes_df['LONG (E)'][i]} {earthquakes_df['LAT (N)'][i]})" for i in range(len(earthquakes_df))]
earthquakes_df

,DATE,TIME(GMT),LAT (N),LONG (E),DEPTH(km),MAGNITUDE(Local),wkt
0,01/01/2021,00:38:24.300000,38.3894,21.9832,8,1.2,POINT (21.9832 38.3894)
1,01/01/2021,00:57:47.900000,38.3748,22.0290,8,0.8,POINT (22.029 38.3748)
2,01/01/2021,01:09:25.700000,38.3693,22.0290,7,1.6,POINT (22.029 38.3693)
3,01/01/2021,01:28:31.000000,40.4311,21.1546,17,2.1,POINT (21.1546 40.4311)
4,01/01/2021,01:53:23.600000,34.9503,24.3283,45,1.9,POINT (24.3283 34.9503)
5,01/01/2021,02:05:36.100000,38.3835,21.9992,10,1.2,POINT (21.9992 38.3835)
6,01/01/2021,02:06:34.900000,38.3775,21.9868,8,1.1,POINT (21.9868 38.3775)
7,01/01/2021,02:07:45.800000,38.3665,22.0226,14,1.1,POINT (22.0226 38.3665)
8,01/01/2021,02:07:47.000000,38.6710,24.0024,16,0.9,POINT (24.0024 38.671)
9,01/01/2021,02:24:37.800000,34.2572,25.2191,15,2.7,POINT (25.2191 34.2572)


In [8]:
gs = gpd.GeoSeries.from_wkt(earthquakes_df['wkt'])
gdf = gpd.GeoDataFrame(earthquakes_df, geometry=gs, crs="EPSG:4326")
gdf

,DATE,TIME(GMT),LAT (N),LONG (E),DEPTH(km),MAGNITUDE(Local),wkt,geometry
0,01/01/2021,00:38:24.300000,38.3894,21.9832,8,1.2,POINT (21.9832 38.3894),POINT (21.9832 38.3894)
1,01/01/2021,00:57:47.900000,38.3748,22.0290,8,0.8,POINT (22.029 38.3748),POINT (22.029 38.3748)
2,01/01/2021,01:09:25.700000,38.3693,22.0290,7,1.6,POINT (22.029 38.3693),POINT (22.029 38.3693)
3,01/01/2021,01:28:31.000000,40.4311,21.1546,17,2.1,POINT (21.1546 40.4311),POINT (21.1546 40.4311)
4,01/01/2021,01:53:23.600000,34.9503,24.3283,45,1.9,POINT (24.3283 34.9503),POINT (24.3283 34.9503)
5,01/01/2021,02:05:36.100000,38.3835,21.9992,10,1.2,POINT (21.9992 38.3835),POINT (21.9992 38.3835)
6,01/01/2021,02:06:34.900000,38.3775,21.9868,8,1.1,POINT (21.9868 38.3775),POINT (21.9868 38.3775)
7,01/01/2021,02:07:45.800000,38.3665,22.0226,14,1.1,POINT (22.0226 38.3665),POINT (22.0226 38.3665)
8,01/01/2021,02:07:47.000000,38.6710,24.0024,16,0.9,POINT (24.0024 38.671),POINT (24.0024 38.671)
9,01/01/2021,02:24:37.800000,34.2572,25.2191,15,2.7,POINT (25.2191 34.2572),POINT (25.2191 34.2572)
